In [33]:
import filterpy.kalman
import numpy as np
import pandas as pd
from filterpy.common import Q_discrete_white_noise, Saver
from matplotlib import pyplot as plt
from scipy.linalg import block_diag
from scipy.optimize import least_squares
import numpy.polynomial.polynomial as poly

# Anc_2D = [
#     [6160189.068799788, 235476.2946930449],
#     [6160105.440872673, 235527.55777198478],
#     [6160007.2800598815, 235462.62533516675],
# ]
Anc_2D = [
    [6160189.068799788, 235476.2946930449],
    [6160162.550425654, 235499.53862634592],
    [6160158.070704494, 235459.50499518885]
]

BASE_STATION_COORDS = (6160188.9135, 235476.0090)
# LR_POLY3 = poly.Polynomial([1.70667308,  0.05036929,  0.91299066,  0.25358677, -1.04010403], [2.816559, 187.039706])
# LR_POLY = poly.Polynomial([0.1], [100, 150.881626])
LR_POLY = poly.Polynomial([ 0.51075268, -0.00117379, -0.43347423,  0.54042526,  0.38338424, -0.33350059, -0.14380512], [0, 100])
# LR_POLY = poly.Polynomial([0])

Anc_2D = [
    [coord[0] - BASE_STATION_COORDS[0], coord[1] - BASE_STATION_COORDS[1]] for coord in Anc_2D
]

# x, y = sym.symbols("x,y")
initial = (-60, 15)
time_filter = ("04:05:40", "04:06:40")
data_source = "2024-07-05 14:03:00.171006"


# dir_fp = "../results/v2/2024-07-05 15:02:00.308097/"
dir_fp = f"../results/v2/{data_source}/"
headers = lambda x: ["dt", f"dist{x}"]
dtypes = lambda x: {"dt": float, f"dist{x}": float}
# parse_dates = ["anc1_dt", "anc0_dt", "anc2_dt", "gnss_dt"]
anc0 = pd.read_csv(dir_fp + "uwb-0.csv", delimiter=',', names=headers(0), dtype=dtypes(0))
anc1 = pd.read_csv(dir_fp + "uwb-1.csv", delimiter=',', names=headers(1), dtype=dtypes(1))
anc2 = pd.read_csv(dir_fp + "uwb-2.csv", delimiter=',', names=headers(2), dtype=dtypes(2))

In [34]:


anc0.set_index('dt', inplace=True)
anc1.set_index('dt', inplace=True)
anc2.set_index('dt', inplace=True)

anc0.index = pd.to_datetime(anc0.index, unit='s').round('50ms')
anc1.index = pd.to_datetime(anc1.index, unit='s').round('50ms')
anc2.index = pd.to_datetime(anc2.index, unit='s').round('50ms')

# time_filter = ("05:08:00", "05:09:30")

gnss_data = pd.read_csv(dir_fp + "gnss.csv", delimiter=',', names=["dt", "northing", "easting"], dtype={"dt": float, "northing": float, "easting": float})
gnss_data.set_index('dt', inplace=True)
gnss_data.index = pd.to_datetime(gnss_data.index, unit='s').round('50ms')
gnss_data = gnss_data.between_time(*time_filter)
gnss_data["northing"] = gnss_data["northing"] - BASE_STATION_COORDS[0]
gnss_data["easting"] = gnss_data["easting"] - BASE_STATION_COORDS[1]


loaded_df = anc0.join(anc1, how='outer').join(anc2, how='outer').join(gnss_data, how='outer')
loaded_df = loaded_df.between_time(*time_filter)
loaded_df = loaded_df.resample(rule='50ms').mean()
# anc0.resample(rule='50ms').mean()
# anc1.resample(rule='50ms').mean()
# anc2.resample(rule='50ms').mean()



In [35]:
df2 = loaded_df.resample(rule='s').count()